# Regridding with [xESMF](https://github.com/pangeo-data/xESMF)

In [ ]:
import os
os.environ['PROJ_LIB'] = "/work/bm0021/conda-envs-public/clisops/share/proj"

%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import xesmf as xe
from pathlib import Path
from clisops.ops.subset import subset

## Prepare data


### Input data


We select two datasets:

*  near surface air temperature (2D) on a rectilinear grid
*  sea surface temperature (2D) on a curvilinear grid

Before creating the remapping weights, we subselect the region around Europe. For that we will use clisops, 
since xarray cannot subset variables with 2D coordinate variables by their value. Also xarray cannot automatically wrap
longitude axis when subsetting across the 0-meridian (or in general: across the easter and western bounds of the grid).

In [ ]:
cmip_root = "/work/ik1017/CMIP6/data"

# near surface air temperature
path_tas = Path(cmip_root,  "CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1/Amon/tas/gn/v20190710/")
path_tas = Path(path_tas, "tas_Amon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_185001-185412.nc")
ds_tas_global = xr.open_dataset(path_tas)

# ocean surface temperature
path_tos = Path(cmip_root,  "CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1/Omon/tos/gn/v20190710/")
path_tos = Path(path_tos, "tos_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_185001-185412.nc")
ds_tos_global = xr.open_dataset(path_tos)

In [ ]:
# Subset near surface air temperature
# area is specified as a tuple (lon_lower, lat_lower, lon_upper, lat_upper)
ds_tas = subset(ds=ds_tas_global, area=(-30, 20, 65, 75), output_type="xarray")[0]

# Plot
ds_tas.tas.isel(time=0).plot.pcolormesh(x="lon", y="lat", shading="auto", cmap="RdBu_r");

In [ ]:
# Subset sea surface temperature
# area is specified as a tuple (lon_lower, lat_lower, lon_upper, lat_upper)
ds_tos = subset(ds=ds_tos_global, area=(-30, 20, 65, 75), output_type="xarray")[0]

# Plot
ds_tos["tos"].isel(time=0).plot.pcolormesh(x="longitude", y="latitude", shading="auto",
                                           cmap="RdBu_r", vmin = -1, vmax=25, 
                                           xlim=(-25,60), ylim=(35,75));

### Target grid

We use xESMF to create a regular Grid with `1.0°x1.0°` resolution.
A little smaller domain is selected for the output, 
to avoid problems like missing values at the domain bounds.



In [ ]:
# Create a regional regular latitude longitude grid of 1 degree resolution
#  xe.util.cf_grid_2d(lon0_b, lon1_b, d_lon, lat0_b, lat1_b, d_lat)
ds_out = xe.util.cf_grid_2d(-25, 60, 1, 35, 70, 1)

# alternatively create a global grid:
#ds_out = xe.util.grid_global(d_lat=1, d_lon=1, cf=True)

# altneratively use clisops:
#from clisops.core.regrid import Grid
#ds_out = Grid(grid_instructor=(-25,60,1,35,70,1)).ds
#ds_out = Grid(grid_instructor=1)
#ds_out = Grid(grid_id="1deg").ds

# display ds_out
ds_out

## Perform regridding

We create the regridding weights by calling `xe.Regridder(grid_in, grid_out, method)`. A `grid` is just an
xarray `DataSet` containing `latitude` and `longitude` coordinates.

xESMF supports multiple remapping methods, but [which method should be applied for the problem](https://climatedataguide.ucar.edu/climate-data-tools-and-analysis/regridding-overview)?

<table style="align:left !important; margin-left:0 !important;">
<thead>
  <tr>
    <th >Method</th>
    <th >Description</th>
    <th >Application</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>bilinear</td>
    <td><ul><li>linear interpolation</li><li>monotonic (i.e. creates no new maxima/minima)</li></ul></td>
    <td>smoothly varying variables (eg. temperature)</td>
  </tr>
  <tr>
    <td>conservative</td>
    <td><ul><li>preserving area average</li><li>maintaining eg. energy budgets</li><li>monotonic</li></ul></td>
    <td><ul><li>discontinuous variables / fluxes</li><li>upscaling (remapping to coarser grid)</li></ul> </td>
  </tr>
  <tr>
    <td>nearest neighbour</td>
    <td>monotonic</td>
    <td>categorical data (eg. binary masks, land usage type)</td>
  </tr>
  <tr>
    <td>patch</td>
      <td><ul><li>least square fit of surrounding surface patches</li><li>preserving derivatives<li</td>
    <td>eg. wind speed</td>
  </tr>
</tbody>
</table>


In [ ]:
# Calculation of weights, using xesmf.Regridder
method_list = ["bilinear",
               "conservative",
               "conservative_normed",
               "patch",
               "nearest_s2d",
               "nearest_d2s"]

regridder = {}
for method in method_list:
    regridder[method+"_tas"] = xe.Regridder(ds_tas, ds_out, method)
    regridder[method+"_tos"] = xe.Regridder(ds_tos, ds_out, method)

In [ ]:
# Applying the weights to remap the data
for method in method_list:
    ds_out[method+"_tas"] = regridder[method+"_tas"](ds_tas.tas.isel(time=0))
    ds_out[method+"_tos"] = regridder[method+"_tos"](ds_tos.tos.isel(time=0))

## Plot the results

The regridding result is consistent with the original data, with a much finer
resolution:


In [ ]:
fig, axes = plt.subplots(3, 2, figsize=[16, 16])

for i, method in enumerate(method_list):
    ax = axes.flatten()[i]
    ds_out[method+"_tas"].plot.pcolormesh(ax=ax, shading="auto", cmap="RdBu_r")
    ax.set_title(method, fontsize=15)

plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=[16, 16])

for i, method in enumerate(method_list):
    ax = axes.flatten()[i]
    ds_out[method+"_tos"].plot.pcolormesh(ax=ax, shading="auto", cmap="RdBu_r")
    ax.set_title(method, fontsize=15)

plt.tight_layout()

*  The missing values of the land cells are contaminating the result, since a target cell will get a missing value 
as long as only a single contributing cell of the source grid has a missing value.
*  `nearest_s2d` maps each cell of the target grid to one cell of the source grid.
*  `nearest_d2s` maps each cell of the source grid to a cell of the target grid. In case of multiple matches, which is usually the case for upscaling, the values will be added up.

## Masking

### Use an input mask to obtain a better result

In [ ]:
# Create the mask - it has to be labeled "mask"
import numpy as np
ds_tos["mask"] = xr.where(~np.isnan(ds_tos["tos"].isel(time=0)), 1, 0)
ds_tos["mask"].plot.pcolormesh(x="longitude", y="latitude", shading="auto",
                               cmap="binary", xlim=(-25,60), ylim=(35,75));

In [ ]:
# Create the weights
for method in method_list:
    regridder[method+"_tos_mask"] = xe.Regridder(ds_tos, ds_out, method)

In [ ]:
# Regrid the data
for method in method_list:
    ds_out[method+"_tos_mask"] = regridder[method+"_tos_mask"](ds_tos.tos.isel(time=0))

In [ ]:
# Plot
fig, axes = plt.subplots(3, 2, figsize=[16, 16])

for i, method in enumerate(method_list):
    ax = axes.flatten()[i]
    ds_out[method+"_tos_mask"].plot.pcolormesh(ax=ax, shading="auto", cmap="RdBu_r")
    ax.set_title(method, fontsize=15)

plt.tight_layout()

*  conservative: the values near the coastline are too low since masked cells contribute to the area average with value 0
*  conservative_normed: here a renormalization is performed for these cells
*  nearest_s2d: cells on land have the same value as the closest unmasked source grid cell

### Instead of an input mask, apply adaptive masking

The `skipna` parameter (boolean, default: False) will cause cells with missing values to be ignored.
Additionally, a renormalization is conducted. Adaptive masking has no effect for the nearest neighbour remapping methods.

The parameter `na_thres` (numeric, default: 1.0) 
allows to specify a maximum ratio of occuring missing values, up to which to apply this *adaptive masking* technique.
If set to 1.0, no target grid cell that has any contribution of a souce grid cell will be set to missing_value. In rare cases this can lead to weird results for single cells with the bilinear and patch remapping methods.
If set to 0.0, the target grid cell will be set to missing_value if any contributing cell of the source grid is missing. 

A big advantage over the input mask approach is, that weights can be reused for input data on the same grid if it is masked differently
(eg. 3D ocean data with the land sea mask varying with depth, 3D atmospheric data with orography varying with height, time dependent masks, ...).

In [ ]:
# We reuse the weights created above and store the remapped datasets in the same output dataset
for method in method_list:
    ds_out[method+"_tos_skipna"] = regridder[method+"_tos"](ds_tos.tos.isel(time=0), skipna=True, na_thres=0.3)

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=[16, 16])

for i, method in enumerate(method_list):
    ax = axes.flatten()[i]
    ds_out[method+"_tos_skipna"].plot.pcolormesh(ax=ax, shading="auto", cmap="RdBu_r")
    ax.set_title(method, fontsize=15)

plt.tight_layout()

The adaptive masking technique is best applied for the conservative and patch remapping methods, 
especially when upscaling (remapping to a coarser grid). 

If `skipna=True, na_thres=1.0` are set, the conservative method yields the same result as the conservative_normed method 
with a defined input mask.

## Regridding of the global dataset

If you tried to remap the global sea surface temperature dataset you will have noticed that the weights generation
is not possible for most of the methods. This is because the dataset still contains the grid halo, 
which are duplicated rows or columns at the edge of the grid, which are necessary during the model simulation, but not so much
in the final data product. On top of that, many grid halos have collapsing bounds (i.e. collapsing on a line or point) and this 
causes (x)ESMF to crash when calculating the weights.

The CDO operator `verifygrid` is very useful in detecting duplicated and collapsing cells.

The xESMF Regridder features the parameter `ignore_degenerate` to deal with collapsing cells, but when using the patch or conservative methods,
the halo would still distort the results, since the halo cells are added up. 
One can use xarray to remove the halo or the input mask option of xESMF to mask the halo (see eg. [this notebook](https://nbviewer.jupyter.org/github/roocs/regrid-prototype/blob/main/docs/notebooks/xESMF_Behaviour_Halo.ipynb) ).

In [ ]:
# Display the dataset
ds_tos_global

In [ ]:
# Let us take a look at the grid, the halo can be spotted:
plt.figure(figsize=(10, 6))
plt.scatter(x=ds_tos_global["longitude"], y=ds_tos_global["latitude"], s=0.001);

In [ ]:
# Use cdo verifygrid to confirm duplicated and/or collapsing cells
! /work/bm0021/conda-envs-public/guidetoclimatedata/bin/cdo verifygrid {path_tos}

In [ ]:
# The halo of the MPIOM GR15 grid (MPI-ESM1-2-LR) consists of the first and last column, 
#  the halo of the MPIOM TP04 grid (MPI-ESM1-2-HR) consists of the first and last column as well as the first two rows
#  (in case it is not obvious as in the latter case, one would have to ask the modelers how their halo is defined ...)
ds_tos_nohalo = ds_tos_global.isel(i=slice(1,801), j=slice(2,405))
ds_tos_nohalo

In [ ]:
# Use cdo selindexbox,lon_idx1,lon_idx2,lat_idx1,lat_idx2 (indexing starts at 1!)
#  to remove halo and verifygrid to confirm duplicated and/or collapsing cells have been removed
! /work/bm0021/conda-envs-public/guidetoclimatedata/bin/cdo verifygrid -selindexbox,2,801,3,404 {path_tos}

Now one can remap the global dataset:

In [ ]:
# Create global target grid
ds_out_global = xe.util.grid_global(d_lat=1, d_lon=1, cf=True)

In [ ]:
# Create the remapping weights (taking a while to calculate)
#  for global datasets, we need to set the periodic parameter, or we might get missing values
#  contaminating the result where the east and west edges of the grid meet, or at the poles.
for method in method_list:
    regridder[method+"_tos_global"] = xe.Regridder(ds_tos_nohalo, ds_out_global, method, periodic=True)
    #In case one wants to see the halo effects:
    #regridder[method+"_tos_global"] = xe.Regridder(ds_tos_global, ds_out_global, method, periodic=True, ignore_degenerate=True)

In [ ]:
# Conduct the remapping
for method in method_list:
    ds_out_global[method+"_tos_global_skipna"] = regridder[method+"_tos_global"](ds_tos_nohalo.tos.isel(time=0), skipna=True, na_thres=0.3)
    # In case one wants to see the halo effects (the proper weights must have been generated in the cell above):
    #ds_out_global[method+"_tos_global_skipna"] = regridder[method+"_tos_global"](ds_tos_global.tos.isel(time=0), skipna=True, na_thres=0.3)

In [ ]:
# Plot
fig, axes = plt.subplots(3, 2, figsize=[16, 12])

for i, method in enumerate(method_list):
    ax = axes.flatten()[i]
    ds_out_global[method+"_tos_global_skipna"].plot.pcolormesh(ax=ax, shading="auto", cmap="RdBu_r")
    ax.set_title(method, fontsize=15)

plt.tight_layout()

## More Examples

For more examples see:

*  the extensive [xESMF documentation](https://xesmf.readthedocs.io/en/latest/)
*  this selection of jupyter notebooks:
   *  [Remapping climate data with xESMF](https://nbviewer.jupyter.org/github/Ouranosinc/pavics-sdi/blob/master/docs/source/notebooks/regridding.ipynb) provided by [Ouranos](https://www.ouranos.ca/en/)
   *  [Remapping regional datasets](https://nbviewer.org/github/roocs/regrid-prototype/blob/main/docs/notebooks/xESMF_curvilinear_nearest_mask_unmapped.ipynb)
   *  [Remapping from unstructured grids](https://nbviewer.org/github/roocs/regrid-prototype/blob/main/docs/notebooks/xESMF_unstructured_locstream_ref.ipynb)
   *  [Regridding with clisops powered by xESMF](https://nbviewer.org/github/roocs/clisops/blob/regrid-main/notebooks/regrid.ipynb)
   *  [Pitfall grid halos when remapping](https://nbviewer.jupyter.org/github/roocs/regrid-prototype/blob/main/docs/notebooks/xESMF_Behaviour_Halo.ipynb)